# **Spatial typology to identify food and nutrition security bottlenecks in Niger**

**Arable land computation**


```bash
Author : Aboubacar HEMA
Supervision : Wim MARIVOET
Contact : a.hema@cgiar.org / w.marivoet@cgiar.org
Role : Research Analyst at IFPRI
Year : 2022
```

-----------------
## **Objective**
-----------------

- To identify arable land in country of interest at 30m resolution, comprising the following steps: 
1. Aggregate Cleared Forest and Cropland datasets 
2. Masking slope > 15%
3. permanent water
4. imprevious surfaces 


-----------------
## **Output**
-----------------



-----------------
## **Dataset**
-----------------


-----------------
## **Technical requirements**
-----------------


```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install geemap
pip install ee
```


-----------------
## **Import the necessary libraries**
-----------------


In [1]:
import ee
import geemap

# Basic libraries of python for numeric and dataframe computations
import numpy as np                              
import pandas as pd

# Basic library for data visualization
import matplotlib.pyplot as plt 

# Slightly advanced library for data visualization            
import seaborn as sns 

In [2]:
# Authenticate and Initialize Earth Engine
geemap.ee_initialize()

-----------------
## **Load data**
-----------------